In [1]:
#https://developers.arcgis.com/python/guide/working-with-feature-layers-and-features/

import os
import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sns
import json
import requests
import matplotlib.pyplot as plt
import bs4

import sklearn
from sklearn import cluster
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

from arcgis import GIS, features, GeoAccessor, GeoSeriesAccessor, geometry
from arcgis.features import SpatialDataFrame
import arcgis

#url = 'https://<insert your Cityworks or AGOL Feature Service>/FeatureServer/2/query?f=json&returnGeometry=true&spatialRel=esriSpatialRelIntersects&maxAllowableOffset=38&inSR=102100&outFields=*&outSR=102100'

url = 'https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services'

print('setup complete')

setup complete


In [2]:
#get list of feature layers
#https://automatetheboringstuff.com/2e/chapter12/

res = requests.get('https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services')
try:
    res.raise_for_status()
except Exception as exc:
    print('There was a problem: %s' % (exc))
print(res.text[:10000])

<html lang="en">
<head>
  <title>Folder: /</title>
  <link href='/ESRI.ArcGIS.SDS.REST.css' rel='stylesheet' type='text/css'>
  <link rel='SHORTCUT ICON' href='/favicon.ico'>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/> 
  <meta name="keywords" content="ArcGIS REST Services Directory Root">
</head>
<body>
<table width="100%" class="userTable">
<tr>
<td class="titlecell">ArcGIS REST Services Directory</td>
</tr>
</table>
<table class="navTable" width="100%">
<tbody>
<tr valign="top">
<td class="breadcrumbs">
<a href="/v400IkDOw1ad7Yad/ArcGIS/rest/services">Home</a> > <a href="/v400IkDOw1ad7Yad/ArcGIS/rest/services">services</a> </td>
<td align="right">
<a href="http://resources.arcgis.com/en/help/arcgis-rest-api/" target="_blank">API Reference</a></td>
</tr>
</tbody>
</table>
<table>
<tr>
<td class="apiref">
<a href="?f=pjson" target="_blank">JSON</a>
</td>
</tr>
</table>
<div class='restHeader'>
<h2>Folder: /</h2>
</div>
<div class='restBody'>
<b>Current Version

In [3]:
bs4AGOL = bs4.BeautifulSoup(res.text, 'html.parser')
AGOLelement = bs4AGOL.select('li > a')
AGOLelement[0].getText()

'1_5_Mile_Radius_Battalions_Current'

In [4]:
len(AGOLelement)

877

In [5]:
print(AGOLelement)

[<a href="/v400IkDOw1ad7Yad/ArcGIS/rest/services/1_5_Mile_Radius_Battalions_Current/FeatureServer">1_5_Mile_Radius_Battalions_Current</a>, <a href="/v400IkDOw1ad7Yad/ArcGIS/rest/services/12_30_2019_PrecDNR/FeatureServer">12_30_2019_PrecDNR</a>, <a href="/v400IkDOw1ad7Yad/ArcGIS/rest/services/2009_2019_update_tiles/FeatureServer">2009_2019_update_tiles</a>, <a href="/v400IkDOw1ad7Yad/ArcGIS/rest/services/2010_2019/FeatureServer">2010_2019</a>, <a href="/v400IkDOw1ad7Yad/ArcGIS/rest/services/2010ACSData/FeatureServer">2010ACSData</a>, <a href="/v400IkDOw1ad7Yad/ArcGIS/rest/services/2019_ChangeMap_WFL1/FeatureServer">2019_ChangeMap_WFL1</a>, <a href="/v400IkDOw1ad7Yad/ArcGIS/rest/services/2019_Plan_update_WFL1/FeatureServer">2019_Plan_update_WFL1</a>, <a href="/v400IkDOw1ad7Yad/ArcGIS/rest/services/2020_Final_Street_Resurfacing_View/FeatureServer">2020_Final_Street_Resurfacing_View</a>, <a href="/v400IkDOw1ad7Yad/ArcGIS/rest/services/2020_Raleigh_Change_Map/FeatureServer">2020_Raleigh_Cha

In [6]:
for FeatureServer in AGOLelement:
    print(FeatureServer.getText())

1_5_Mile_Radius_Battalions_Current
12_30_2019_PrecDNR
2009_2019_update_tiles
2010_2019
2010ACSData
2019_ChangeMap_WFL1
2019_Plan_update_WFL1
2020_Final_Street_Resurfacing_View
2020_Raleigh_Change_Map
2020_Rejuvenation_Contract_Viewer
2021_Street_Resurfacing_V1
2050_MTP_Local_Plans_Review_Data
56e7ae324b4547deae2d76246ff6c38b_view_1576068134059
a1dc29
AADT2017Avg_BlockGroups
AAS_Program_Data
Abbotts_Creek_Park
Abbotts_Creek_Participants_WFL1
ACS_2018_Hispanic_or_Latino_By_Tract
ACS_2018_Hispanic_or_Latino_By_Tract_Raleigh
Active_Road_Closures_Florence
ADA_Parking_Spaces_Proposed
Adopt_App_Users
AdoptAStops_public
Adopted_Capital_Projects
Adopted_Expense_Budget
Adopted_Expense_Budget_Current_Fiscal_Year
Adopted_Revenue_Budget
Adopted_Revenue_Budget_Current_Fiscal_Year
AdoptionAddresses Density
AerialLighting
AffordableHousing_Geocoded
All Dog Friendly Apartments Density
All Events Density
Alley_Inventory
Annexation_Agreement_Properties
Annexation_History_1792_to_2009
AnnexationAgreements

survey123_569215b560f940a3993eb356c6cd5f61_fieldworker
survey123_5d7f4fab941948cfb8d64649c67477d4_fieldworker
survey123_64537776f70a4376bd7758cbf37869de_fieldworker
survey123_661b7bfc4a8f477791a949049f5f78ea_fieldworker
survey123_66d5553e5489489b86e181a512bcf387_fieldworker
survey123_692420d022c24654bdb9464a17be759d_fieldworker
survey123_6a51a3fcf12c4c7ba872b2d06a706641_fieldworker
survey123_6e0090984ba34d4d96503d84bd3e0580_fieldworker
survey123_6e6c86f3f9e14165a3743d36cf17a262_fieldworker
survey123_6e6c86f3f9e14165a3743d36cf17a262_stakeholder
survey123_72eb4789a8014bbbaf8ac35532a76844_fieldworker
survey123_746578b93b284180af3f86e4ab9b98f7_fieldworker
survey123_7d5ed6536d134748aabbc0f2f38b267c_fieldworker
survey123_7f7afa04d63e4c09b34c1afd9b4549d9_fieldworker
survey123_83ca2356c20a45bdb0183eef755cc5d5_fieldworker
survey123_84e22a51561e4d538561573704d4a633_fieldworker
survey123_8724104388fd4b4fb847d8e6ba52b852
survey123_8724104388fd4b4fb847d8e6ba52b852_fieldworker
survey123_872e4889206a

In [7]:
for FeatureServer in AGOLelement:
    print(str(url) + "/" + str(FeatureServer.getText()) + "/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson")

https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/1_5_Mile_Radius_Battalions_Current/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/12_30_2019_PrecDNR/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/2009_2019_update_tiles/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/2010_2019/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/2010ACSData/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/2019_ChangeMap_WFL1/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/2019_Plan_update_WFL1/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.ar

https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/RandleighTract/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/RCAPs/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/RCC_Map_WFL1/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/RCPAC_WFL1/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/Recreation_Features/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/RecreationDistricts/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/Recycle_Line/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/servi

In [8]:
# this did not work
def replaced(sequence, old, new):
    return (new if x == old else x for x in sequence)

new_words = list(replaced(AGOLelement, ' ', '%20'))

print(new_words[32])



<a href="/v400IkDOw1ad7Yad/ArcGIS/rest/services/All%20Dog%20Friendly%20Apartments%20Density/FeatureServer">All Dog Friendly Apartments Density</a>


In [10]:
# this does work for Feature Services with ' ' in their names
ServiceList = []
for FeatureServer in new_words:
    FS = str(url) + "/" + str(FeatureServer.getText()).replace(' ','%20') + "/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson"
    print(FS)
    ServiceList.append(FS)

https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/1_5_Mile_Radius_Battalions_Current/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/12_30_2019_PrecDNR/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/2009_2019_update_tiles/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/2010_2019/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/2010ACSData/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/2019_ChangeMap_WFL1/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/2019_Plan_update_WFL1/FeatureServer/0/query?where=1%3D1&outFields=*&f=pjson
https://services.ar

In [ ]:
# Ok, now I have a list of all feature services, if they have a /0 layer...
# there was probably an easier way to do this via the ArcGIS Python APi
# but this was good practice with web scraping

for url in ServiceList:
    resp = requests.get(url=url)
    data = resp.json()
    if str(data) != "{'error': {'code': 400, 'message': 'Invalid URL', 'details': ['Invalid URL']}}":
        dicts = [{k:v} for k,v in data.items()]
        # List Type, but it's a list of dictionaries
        #type(data['features'])
        PRCRCityworks = pd.DataFrame(data['features'])
        print(PRCRCityworks.shape)
        # Converting List of Dictionaries to a pandas dataframe
        PRCR2 = pd.DataFrame(d['attributes'] for d in data['features'])
        #print(PRCR2)
        PRCR2.head()

(28, 2)
(2000, 2)
(1038, 2)
(252, 2)
(1, 2)
(1, 2)
(2000, 2)
(138, 2)
(51, 2)
(6, 2)
(584, 2)
(2000, 2)
(455, 2)
(127, 2)
(201, 2)
(150, 2)
(187, 2)
(96, 2)
(15, 2)
(16, 2)
(221, 2)
(365, 1)
(2000, 1)
(2000, 1)
(2000, 1)
(453, 1)
(137, 2)
(50, 2)
(10, 2)
(46, 2)
(25, 2)
(15, 2)
(636, 2)
(6, 2)
(285, 2)
(1, 2)
(45, 2)
(12, 2)
(154, 2)
(156, 2)
(2, 2)
(11, 2)
(11, 2)
(1137, 2)
(28, 2)
(23, 2)
(55, 2)
(32, 2)
(97, 2)
(342, 2)
(1, 2)
(19, 2)
(150, 2)
(17, 2)
(34, 2)
(1674, 2)
(2000, 2)
(1, 2)
(107, 2)
(107, 2)
(43, 2)
(9, 2)
(9, 2)
(2000, 2)
(2000, 2)
(2000, 2)
(418, 2)
(2000, 2)
(3, 2)
(16, 2)
(234, 2)
(3014, 2)
(39, 2)
(35, 2)
(0, 0)
(1, 2)
(1, 2)
(2000, 2)
(1, 2)
(1336, 2)
(154, 2)
(187, 2)
(3, 2)
(183, 2)
(35, 2)
(16, 2)
(20, 2)
(1, 2)
(13, 2)
(54, 2)
(2, 2)
(22, 2)
(1, 2)
(690, 2)
(39, 2)
(205, 2)
(2, 2)
(87, 2)
(3, 2)
(6, 2)
(2000, 2)
(33, 2)
(6, 2)
(1, 2)
(31, 2)
(10000, 2)
